# 导包

In [1]:
# -*- coding: utf-8 -*-
import json
import re
import math
import os
import threading
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras
from keras.utils import plot_model
from sklearn.model_selection import train_test_split

!pip install pyts==0.7.1


from extract_csi import extract_csi, get_scaled_csi
# data_crop, down_sampling, moving_average, add_jitter, add_scaling
from get_files_info import *


!cat /proc/cpuinfo | grep 'model name' |uniq
!nvidia-smi
# 设置GPU内存按需分配
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

model name	: Intel(R) Core(TM) i9-10900F CPU @ 2.80GHz
Mon Dec 21 09:29:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 455.45.01    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3070    Off  | 00000000:01:00.0 Off |                  N/A |
|  0%   43C    P3    36W / 270W |    286MiB /  7979MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                 

In [2]:
raw_file_path = r'../data_qinghua/20181112/user2' 
save_path = r'../data_processed'

# 特征工程

## 提取CSI中想要的部分

In [3]:

# 提取CSI中想要的部分
# def process_data(raw_file_path, save_path):
#     X = {}
#     thread_list= []
#     for f in tqdm(os.listdir(raw_file_path)):             
#         if f.endswith('.dat'):
#             def processor():
#                 file_name = os.path.join(raw_file_path, f)
#                 extracted_data = extract_csi(file_name)
#                 # print('processing {} the length of this file is:{}'.format(file_name, len(extracted_data)))
#                 tx, rx, sub = extracted_data[0]['csi'].shape
#                 data_csi = np.zeros((len(extracted_data), tx, rx, sub), dtype=np.complex64)
#                 # qinghua (1938, 1, 3, 30)  ;    self
#                 for i in range(len(extracted_data)):
#                     data_csi[i] = get_scaled_csi(extracted_data[i])
#                 data_csi = np.clip(np.abs(np.squeeze(data_csi)), 1e-8, 1e100).reshape(-1, tx*rx, 30)   
#                 data = np.zeros((data_csi.shape[0], tx*rx))  # N*4
#                 for ant in range(tx*rx):  # 每个天线对上的CSI变化趋势相同,为节约计算这里选择天线对即可
#                     data_csi_ant = data_csi[:, ant, :]
#                     b, a = signal.butter(5, 4*2/30, 'low')
#                     var_max = 0
#                     s_max = None
#                     for s in range(30):
#                         carrier_data = signal.lfilter(
#                             b, a, data_csi_ant[:, s])  # N*1
#                         length = len(carrier_data)
#                         var_temp = np.var(carrier_data[length//5:3*length//5])
#                         if var_max < var_temp:
#                             var_max = var_temp
#                             s_max = carrier_data
#                     data[:, ant] = s_max
#                 scio.savemat(os.path.join(save_path, f.split('.')[0]+'.mat'), {'csi': data})
#                 X[f] = data
#             t = threading.Thread(target=processor, args=())                              
#             t.setDaemon(True)  # 设置为守护线程                              
#             thread_list.append(t)
            
#     for t in thread_list:         
#         t.start()  # 启动线程     
#     for t in thread_list: 
#         t.join()  # 等待子线程结束
#     print('-----------------------------------all raw file processed---------------------------------')
#     # return X
# process_data(raw_file_path, save_path)

## 数据信息生成

In [4]:
train_file_path = save_path

train_label_file_list, train_wav_file_list = get_file_list(train_file_path)

train_label_file_list, test_label_file_list, train_wav_file_list, test_wav_file_list = train_test_split(train_label_file_list, train_wav_file_list, test_size=0.1, random_state=42)

train_label_list = get_label_data(train_label_file_list)

# 用迭代器的时候需要
# train_wav_file_list, validate_wav_file_list, train_label_list, validate_label_list = train_test_split(train_wav_file_list, train_label_list, test_size=0.2, random_state=0)

# 每个文件拼音标签的集合
with open('label_list.txt', 'w') as f:  
    f.write('\n'.join(train_label_list))  
py_list = gen_py_list(train_label_list)  # 拼音的集合

with open('pinyin_list.txt', 'w') as f:
    f.write('\n'.join(py_list))  # 保存拼音列表

train_file_nums = len(train_wav_file_list)
# validate_file_nums = len(validate_wav_file_list)

py_list_size = len(py_list)  # 模型输出的维度
print('py_list:', py_list)
print('train files amount:', len(train_label_list), train_file_nums, 'label amount:', py_list_size)#, 'validate files amount:', len(validate_label_list), validate_file_nums,)

# 测试数据


# test_file_path = test_save_path # r'/content/drive/My Drive/data_thchs30'
# test_label_file_list, test_wav_file_list = get_file_list(test_file_path)

test_label_list = get_label_data(test_label_file_list)
print('test files amount:',len(test_label_file_list), len(test_wav_file_list))
test_data_num = len(test_label_file_list)

100%|██████████| 1500/1500 [00:00<00:00, 39056.02it/s]

py_list: ['8', '9', '1', '10', '3', '6', '7', '4', '2', '5', '_']
train files amount: 13500 13500 label amount: 11
test files amount: 1500 1500


# 训练

In [ ]:
from model_crnn import create_model
image_size=64  
batch_size = 16
logs_path = '../logs'

input_size=(None, image_size, 1)
base_model, ctc_model = create_model(input_size, output_size=py_list_size) 
print(ctc_model.summary())

# keras.utils.plot_model(base_model, show_shapes=True)

if not os.path.exists(logs_path):  # 判断保存模型的目录是否存在
    os.makedirs(logs_path)  # 如果不存在，就新建一个，避免之后保存模型的时候炸掉

# train_batch_gen = get_batch_generator(batch_size, train_wav_file_list, train_label_list, py_list, image_size)
# validate_batch_gen = get_batch_generator(batch_size, validate_wav_file_list, validate_label_list, py_list, image_size)
# input_data = next(train_batch_gen)[0]
# plt.imshow(input_data['the_inputs'][0].T[0])
# plt.show()
# print(input_data['the_inputs'].shape, input_data['the_labels'].shape, input_data['input_length'].shape, input_data['label_length'].shape)

train_data = get_train_data(train_wav_file_list, train_label_list, py_list, image_size)

cb = []
cb.append(keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0))
# 当监测值不再改善时，该回调函数将中止训练可防止过拟合
cb.append(keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=50, verbose=1, mode='auto'))
# his = ctc_model.fit_generator(train_batch_gen, verbose=1, steps_per_epoch=train_file_nums//batch_size, validation_data=validate_batch_gen, validation_steps=validate_file_nums//batch_size, epochs=1000, callbacks=cb)  

his = ctc_model.fit(train_data[0], train_data[1], validation_split=0.1, batch_size=128, epochs=1000, callbacks=cb)  # callback的epoch都是对fit里的参数来说

#  保存模型结构及权重
ctc_model.save_weights(r'save_weights.h5')
with open(r'model_struct.json', 'w') as f:
    json_string = base_model.to_json()
    f.write(json_string)  # 保存模型信息
print('模型结构及权重已保存')

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_inputs (InputLayer)         [(None, None, 64, 1) 0                                            
__________________________________________________________________________________________________
Conv2D_1-1 (Conv2D)             (None, None, 64, 64) 640         the_inputs[0][0]                 
__________________________________________________________________________________________________
BatchNormal_1-1 (BatchNormaliza (None, None, 64, 64) 256         Conv2D_1-1[0][0]                 
__________________________________________________________________________________________________
Conv2D_1-2 (Conv2D)             (None, None, 64, 64) 36928       BatchNormal_1-1[0][0]            
____________________________________________________________________________________________

# 评估

In [ ]:
# 加载权重
with open(r'model_struct.json') as f:
    model_struct = f.read()
test_model = keras.models.model_from_json(model_struct)
test_model.load_weights(r'save_weights.h5')
# model = keras.models.load_model('all_model.h5')
print('模型已加载')
py_list = []
with open(r'pinyin_list.txt', 'r') as f:
    contents = f.readlines()
for line in contents:
    i = line.strip('\n')
    py_list.append(i)
print('py_list已加载')

# 对模型进行评价


def evaluate(kind, wavs, labels):
    data_num = len(wavs)
    error_cnt = 0
    for i in range(data_num):
        pre = test_model.predict(wavs[i])  # (1, 20, 11)
        pre_index, pre_label = decode_ctc(pre, py_list)  # ['5']
        try:
            pre_label = int(pre_label[0])
        except:
            pre_label = None
        label = int(py_list[labels[i]])
        if label != pre_label:
            error_cnt += 1
            print('真实标签：', label, '预测结果', pre_label)
    print('{}:样本数{}错误数{}准确率：{:%}'.format(
        kind, data_num, error_cnt, (1-error_cnt/data_num)))


# 训练集
train_wavs, train_labels = get_test_data(image_size, train_wav_file_list, train_label_list, py_list)
# 测试集
test_wavs, test_labels = get_test_data(image_size, test_wav_file_list, test_label_list, py_list)

evaluate('trian', train_wavs, train_labels)
evaluate('test', test_wavs, test_labels)